In [2]:
import os
import gym
import gym_donkeycar
import numpy as np
import random
import tensorflow
import shimmy
from rich.live import Live
import stable_baselines3
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO

In [3]:
DONKEY_GYM = True
DONKEY_SIM_PATH = "remote"
DONKEY_GYM_ENV_NAME = "donkey-generated-track-v0"
body_style = "car01"
body_rgb = (128, 128, 128)
car_name = "PPO"
font_size = 100
WEB_CONTROL_PORT = int(os.getenv("WEB_CONTROL_PORT", 8887))
port = 9091

In [29]:
folder = '/mnt/d/donkeycar_model/PPO/'

In [30]:
conf = { "DONKEY_GYM":DONKEY_GYM, "DONKEY_SIM_PATH":DONKEY_SIM_PATH, "DONKEY_GYM_ENV_NAME":DONKEY_GYM_ENV_NAME, 
         "body_style":body_style, "body_rgb":body_rgb, "car_name":car_name, "font_size":font_size, 
         "WEB_CONTROL_PORT" : WEB_CONTROL_PORT, "port" : port }

In [79]:
env = gym.make("donkey-minimonaco-track-v0", conf=conf)

INFO:gym_donkeycar.core.client:connecting to localhost:9091 
INFO:gym_donkeycar.envs.donkey_sim:on need car config
INFO:gym_donkeycar.envs.donkey_sim:sending car config.
INFO:gym_donkeycar.envs.donkey_sim:sim started!


starting DonkeyGym env


INFO:gym_donkeycar.envs.donkey_sim:New lap time: 20.86 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 23.42 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 28.29 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 28.64 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 28.03 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 28.08 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 27.55 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 28.8 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 27.94 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 27.95 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 27.97 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 27.99 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 28.64 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 27.68 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 27.91 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 28.76 s

In [80]:
for i in range(1000):
    # preparation
    # if i: 
    #     model = PPO.load(f'{folder}{i*2500}.pth')
    #     Log = np.load(folder+f"Log_PPO-{i*2500}.npy", allow_pickle=True).item()
    if i == 0:
        model = model = PPO("CnnPolicy", env, n_steps=500, verbose=1 )
        Log={"TestReward":[]}

    # training
    obs = env.reset()
    model.learn(total_timesteps=2500, progress_bar=0)
    model.save(f'{folder}{(i+1)*2500}.pth')

    # testing
    tem = 0
    for _ in range(20):
        obs = env.reset()
        while True:
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, info = env.step(action)
            # print (reward, done)
            env.render()
            tem += reward
            if done: break
    Log["TestReward"].append(tem/20)
    print(f"testing {(i+1)*2500} : {tem/20}")
    np.save(f"{folder}/Log_PPO-{(i+1)*2500}.npy", Log)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 69.7     |
|    ep_rew_mean     | 43.6     |
| time/              |          |
|    fps             | 14       |
|    iterations      | 1        |
|    time_elapsed    | 34       |
|    total_timesteps | 500      |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 72.1         |
|    ep_rew_mean          | 50.1         |
| time/                   |              |
|    fps                  | 14           |
|    iterations           | 2            |
|    time_elapsed         | 68           |
|    total_timesteps      | 1000         |
| train/                  |              |
|    approx_kl            | 0.0045110416 |
|    clip_fraction        | 0.15         |
|    cli

KeyboardInterrupt: 

In [51]:
# testing
# model = model = PPO("CnnPolicy", env, n_steps=500, verbose=1 )
# model.learn(total_timesteps=2500, progress_bar=0)

In [86]:
num = 60000
test_model = PPO.load(f"{folder}{num}.pth")
obs = env.reset()
tem = 0
while True:
    action, _states = test_model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    # print (reward, done)
    env.render()
    tem += reward
    if done: break
print(tem)

1343.4235813868684


In [87]:
env.close()